In [ ]:
import open3d as o3d
import os
import numpy as np
os.environ['DISPLAY'] = ":11.0"

In [ ]:

# pcd = o3d.io.read_point_cloud("pcd_obj.ply")
pcd = o3d.io.read_point_cloud("pcd_scene.ply")
print(pcd)
# 法线估计
radius = 0.01  # 搜索半径
max_nn = 30  # 邻域内用于估算法线的最大点数
pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius, max_nn))
o3d.visualization.draw_geometries([pcd], window_name="法线估计",
                                  point_show_normal=True,
                                  width=800,  # 窗口宽度
                                  height=600)  # 窗口高度

In [ ]:
import os
import xml.dom.minidom
import yaml

def RemoveXml_space(xml_path):
    with open(xml_path,"r") as f:
        res = f.readlines() #res 为列表
    res = [x for x in res if x.split()] #将空行从 res 中去掉
    
    with open(xml_path,"w") as f:
        f.write("".join(res))  #将 res 转换为 字符串重写写入到xml
    return True

In [ ]:
input_dir = "experiment/resources/objects/pybullet_ycb"
# with open(f'/home/shixin/data/pybullet_ycb/config.yml','r') as ff:
#     cfg = yaml.safe_load(ff)

for i, file in  enumerate(sorted(os.listdir(input_dir))):
    obj_dir = os.path.join(input_dir, file)
    # ms.load_new_mesh(os.path.join(obj_dir, "textured.obj"))

    # # compute the geometric measures of the current mesh
    # # and save the results in the out_dict dictionary
    # out_dict = ms.get_geometric_measures()

    # (3, )
    # com = out_dict["center_of_mass"]
    # # (3, 3)
    # iner_matrix = out_dict["inertia_tensor"]
    # inertia_xx = iner_matrix[0, 0]
    # inertia_xy = iner_matrix[0, 1]
    # inertia_xz = iner_matrix[0, 2]
    # inertia_yy = iner_matrix[1, 1]
    # inertia_yz = iner_matrix[1, 2]
    # inertia_zz = iner_matrix[2, 2]

    # print(com)

    # mass = cfg['mass'][i]

    # if os.path.exists(os.path.join(obj_dir, "model.xml")):
    #     os.remove(os.path.join(obj_dir, "model.xml"))
    
    # if os.path.exists(os.path.join(obj_dir, "model_tune.urdf")):
    #     os.replace(os.path.join(obj_dir, "model_tune.urdf"), os.path.join(obj_dir, "model.urdf"))

    # if os.path.exists(os.path.join(obj_dir, "model.urdf")):
    #     os.replace(os.path.join(obj_dir, "model.urdf"), os.path.join(obj_dir, "model.xml"))

    RemoveXml_space(os.path.join(obj_dir, "model.xml"))

    if os.path.exists(os.path.join(obj_dir, "model.xml")):
        os.replace(os.path.join(obj_dir, "model.xml"), os.path.join(obj_dir, "model.urdf"))

    
    # os.remove(os.path.join(obj_dir, "model_tune.urdf"))
    # os.replace(os.path.join(obj_dir, "model_tune.urdf"), os.path.join(obj_dir, "model_tune.xml"))
    # xmlDoc = xml.dom.minidom.parse(os.path.join(obj_dir, "model.urdf"))
    # xmlDoc.getElementsByTagName("mass")[0].setAttribute("value", str(mass * 0.001))
    
    # xmlDoc.getElementsByTagName("inertia")[0].setAttribute("ixx", str(1e-3))
    # xmlDoc.getElementsByTagName("inertia")[0].setAttribute("ixy", str(0))
    # xmlDoc.getElementsByTagName("inertia")[0].setAttribute("ixz", str(0))
    # xmlDoc.getElementsByTagName("inertia")[0].setAttribute("iyy", str(1e-3))
    # xmlDoc.getElementsByTagName("inertia")[0].setAttribute("iyz", str(0))
    # xmlDoc.getElementsByTagName("inertia")[0].setAttribute("izz", str(1e-3))
    # com_value = xmlDoc.getElementsByTagName("origin")[0].getAttribute("xyz")
    # print(f"before com value: {com_value}")
    # xmlDoc.getElementsByTagName("origin")[0].setAttribute("xyz", f"{com[0]} {com[1]} {com[2]}")
    # com_value = xmlDoc.getElementsByTagName("origin")[0].getAttribute("xyz")
    # print(f"after com value: {com_value}")
    # os.replace(os.path.join(obj_dir, "model_tune.xml"), os.path.join(obj_dir, "model_tune.urdf"))
    # with open(os.path.join(obj_dir, "model.urdf"), 'wb') as f:
    #     f.write(xmlDoc.toprettyxml(encoding='utf-8'))
    # os.remove(os.path.join(obj_dir, "model_tune.xml"))




In [ ]:

    

RemoveXml_space("experiment/resources/objects/pybullet_ycb/035_power_drill/model.xml")


In [ ]:
student = {'小萌': '1001', '小智': '1002', '小强': '1003', '小明': '1004'}
try:
    list (student.keys()) [list (student.values()).index ('100')]
except:
    print('没有找到对应的值')


In [ ]:
from graspnetAPI.graspnet import GraspNet

In [ ]:
graspnet_train = GraspNet(root="data/Benchmark/graspnet", camera="realsense", split="train")
print(f"训练物体: \n {graspnet_train.objIds}")

graspnet_train = GraspNet(root="data/Benchmark/graspnet", camera="realsense", split="test_seen")
print(f"seen 物体: \n {graspnet_train.objIds}")

graspnet_train = GraspNet(root="data/Benchmark/graspnet", camera="realsense", split="test_similar")
print(f"similar: \n {graspnet_train.objIds}")

graspnet_train = GraspNet(root="data/Benchmark/graspnet", camera="realsense", split="test_novel")
print(f"novel: \n {graspnet_train.objIds}")


In [ ]:
import random

for i in range(8):
    num_objs = random.sample(range(5, 8), 1)
    random_numbers = random.sample(range(11), num_objs[0])
    print(random_numbers)

In [ ]:
# 示例数组，包含 True 和 False 值
array = np.array([True, False, True, True, True, False, True, False])

# 使用 count() 函数统计值为 True 的元素个数
count_true = np.count_nonzero(array)

# 输出值为 True 的元素个数
print("值为 True 的元素个数为:", count_true)

## 可视化指定场景视角的关键点和抓取位姿

In [2]:
import torch
import scipy.io as scio
from models.graspnet import GraspNet, pred_decode
from dataset.graspnet_dataset import GraspNetDataset, collate_fn
from graspnetAPI import GraspGroup, GraspNetEval
from graspnetAPI.utils.utils import generate_scene_model
from graspnetAPI.utils.eval_utils import get_scene_name, create_table_points, parse_posevector, \
    load_dexnet_model, transform_points, compute_point_distance, compute_closest_points, \
        voxel_sample_points, topk_grasps, get_grasp_score, collision_detection, eval_grasp
from utils.data_utils import CameraInfo, transform_point_cloud, \
    create_point_cloud_from_depth_image, get_workspace_mask, remove_invisible_grasp_points
from utils.collision_detector import ModelFreeCollisionDetector
import numpy as np
import open3d as o3d
import os
from experiment.utils import toOpen3dCloud
os.environ['DISPLAY'] = ":11.0"

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


pybullet build time: May 20 2022 19:44:17


In [3]:
TEST_DATASET = GraspNetDataset("data/Benchmark/graspnet", valid_obj_idxs=None, grasp_labels=None, split="test", 
                               camera="realsense", num_points=20000, remove_outlier=True, augment=False, load_label=False)

ge = GraspNetEval(root="data/Benchmark/graspnet", camera="realsense", split="test")
net = GraspNet(input_feature_dim=0, num_view=300, num_angle=12, num_depth=4, 
               cylinder_radius=0.05, hmin=-0.02, hmax_list=[0.01,0.02,0.03,0.04], is_training=False)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
net.to(device)
# Load checkpoint
checkpoint = torch.load("logs/log_rs/2023-09-10-09-44/checkpoint.tar")
net.load_state_dict(checkpoint['model_state_dict'])

Loading data path and collision labels...:   0%|          | 0/90 [00:00<?, ?it/s]

Loading data path...: 100%|██████████| 90/90 [00:00<00:00, 351.57it/s]


<All keys matched successfully>

In [4]:
def get_data(scene_id, img_id, raw=False):
    ret_dict = TEST_DATASET.get_data((scene_id - 100) * 256 + img_id, return_raw_cloud=raw)
    # if raw == False:
    #     pcd = ret_dict['point_clouds']
    #     obj_mask = ret_dict['seg_mask']
    #     pcd_obj_inds = np.argwhere(obj_mask>0).squeeze() # (N_obj,)
    #     pcd_obj = pcd[obj_mask>0]
    #     num_pts_obj = 1024
    #     if len(pcd_obj) >= num_pts_obj:
    #         idxs = np.random.choice(len(pcd_obj), num_pts_obj, replace=False)
    #     else:
    #         idxs1 = np.arange(len(pcd_obj))
    #         idxs2 = np.random.choice(len(pcd_obj), num_pts_obj-len(pcd_obj), replace=True)
    #         idxs = np.concatenate([idxs1, idxs2], axis=0)
    #     pcd_obj_inds = pcd_obj_inds[idxs] # (1024, )
    #     ret_dict['pcd_obj_inds'] = pcd_obj_inds
    
    return ret_dict

In [10]:
# scene_id = 100
img_id = 0

for scene_id in range(103, 106):
    # 点云，颜色，segmentation， 物体点索引
    data = [get_data(scene_id, img_id)]

    # 扩展 batch 维度
    batch_data = collate_fn(data)

    # 送到 GPU
    for key in batch_data:
        if 'list' in key:
            for i in range(len(batch_data[key])):
                for j in range(len(batch_data[key][i])):
                    batch_data[key][i][j] = batch_data[key][i][j].to(device)
        else:
            batch_data[key] = batch_data[key].to(device)

    # Forward pass，保存关键点和场景
    with torch.no_grad():
        end_points = net(batch_data)
        grasp_preds = pred_decode(end_points)

    vis_result_dir = f"vis_result/{scene_id}_{img_id}"
    if not os.path.exists(vis_result_dir):
        os.makedirs(vis_result_dir)

    # 抓取位姿送到 CPU
    preds = grasp_preds[0].detach().cpu().numpy()
    gg = GraspGroup(preds)
    num_pred = len(gg)
    # total_pred += num_pred

    # 抓取点 (1024, 3)
    graspable_p = preds[:, 13:16]
    graspable_pcd_o3d = toOpen3dCloud(graspable_p)
    o3d.io.write_point_cloud(f"{vis_result_dir}/graspable_pcd.ply", graspable_pcd_o3d)

    # 旋转矩阵
    # rot_matrix = preds[:, 4:13]
    # rot_matrix = rot_matrix.reshape((1024, 3, 3))
    # (1024, 3)
    # approach_vector = rot_matrix[:, :, 0]
    # approach_vector_o3d = toOpen3dCloud(approach_vector)
    # o3d.io.write_point_cloud(f"{vis_result_dir}/approach_vector.ply", approach_vector_o3d)

    # 原始规模的点云
    cloud, rgb = get_data(scene_id, img_id, raw=True)
    # 保存到本地
    raw_pcd_o3d = toOpen3dCloud(cloud, rgb)
    o3d.io.write_point_cloud(f"{vis_result_dir}/raw_pcd.ply", raw_pcd_o3d)

    # collision detection 
    # mfcdetector = ModelFreeCollisionDetector(cloud, voxel_size=0.01)
    # collision_mask = mfcdetector.detect(gg, approach_dist=0.05, collision_thresh=0.01)
    # num_coll = np.count_nonzero(collision_mask)
    # # total_coll += num_coll
    # print(f"collision rate: {(num_coll/num_pred):.2f}\n")
    # gg = gg[~collision_mask]

    # 可视化抓取
    # table = create_table_points(1.0, 1.0, 0.05, dx=-0.5, dy=-0.5, dz=-0.05, grid_size=0.008)
    # _, pose_list, camera_pose, align_mat = ge.get_model_poses(scene_id, img_id)
    # table_trans = transform_points(table, np.linalg.inv(np.matmul(align_mat, camera_pose)))
    # t = o3d.geometry.PointCloud()
    # t.points = o3d.utility.Vector3dVector(table_trans)
    # model_list = generate_scene_model("data/Benchmark/graspnet", 'scene_%04d' % scene_id , 
                                    #   img_id, return_poses=False, align=False, camera="realsense")
    nms_gg = gg.nms()
    nms_gg = nms_gg[:5]
    grasps_geometry = nms_gg.to_open3d_geometry_list()
    
    pcd = ge.loadScenePointCloud(scene_id, "realsense", img_id)
    o3d.visualization.draw_geometries([pcd, *grasps_geometry])
    # o3d.visualization.draw_geometries([pcd, *grasps_geometry, *model_list])
    # o3d.visualization.draw_geometries([*grasps_geometry, *model_list, t])


## robotiq-85 gripper 模型

In [ ]:
import FreeCAD as App
import ImportGui
import Mesh

In [ ]:
# 定义Step文件路径
step_file = "/path/to/your/step/file.step"
# 定义STL文件路径
stl_file = "/path/to/your/stl/file.stl"

# 初始化FreeCAD应用
App.newDocument("Unnamed")

# 导入Step文件
ImportGui.open(step_file)

# 获取导入的Step对象
shape = App.ActiveDocument.ActiveObject.Shape

# 将Step对象转换为Mesh对象
mesh = Mesh.Mesh()
mesh.fromShape(shape)

# 将Mesh对象保存为STL文件
mesh.write(stl_file)